# IMAGE ANALYSIS, IMAGE #2

Libary: OpenCV <br>
Aim: identify roots from binary image, read previous day csv-file and compare IMAGE #2 roots with IMAGE #1 data <br>
ROOT FROM IMAGE #2 IS SAME AS IMAGE #1 ROOT IF IT HAS BIGGER RADIUS AND DISTANCE BETWEEN ROOT CIRCLE'S CENTER IS UNDER 30 (CRITERIA 30 CAN BE (SHOULD BE) ADJUSTED) 

In [14]:
from matplotlib import pyplot as plt
import numpy as np
import cv2
from ipywidgets import *
from IPython.display import clear_output, display
import math

In [15]:
# import DETECT_ROOT funcitons 

from root_functions import scale_crop_gray, edge_areas_mask, change_areas_mask, light_areas_mask_color, remove_isolated_pixels
from root_functions import add_text, grab_contours, sort_contours

In [16]:
from ipywidgets import *
from tkinter import Tk, filedialog
%gui tk

In [17]:
def open_file():
    # Create Tk root
    root = Tk()
    # Hide the main window
    root.withdraw()
    root.call('wm', 'attributes', '.', '-topmost', True)

    infiles = filedialog.askopenfilename(multiple=True)
    return infiles


### Find contours

Find countours and collect contour coordinates to a dictionary for further dataframe generation <br>
If radius value is zero or less than 5, root is not taken

In [18]:
# FIND CONTOURS

def find_contours(filename, image, mask):
    # find the contours in the mask, then sort them from left to right
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = grab_contours(cnts)
    
    # Sort contours from left to right
    cnts = sort_contours(cnts)[0]
    print(f'Contours: {len(cnts)}')

    # loop over the contours and collect coordinates to the dictionary
    coordinates = {}
    imageID = 0
    for c in cnts:
    # get the bright spot of the image
        (x, y, w, h) = cv2.boundingRect(c)
        ((cX, cY), radius) = cv2.minEnclosingCircle(c)
        if int(radius) <= 0:
            pass
        else:
            imageID += 1
            # coordinates[i] = [filename.split("_")[3], filename, (int(cX), int(cY)), int(radius), 'Y']
            coordinates[imageID] = [filename[0].split('/')[-1], (int(cX), int(cY)), int(radius), 0]
    return coordinates

In [19]:
# draw circle around roots and and mark center of the circle

def draw_circles_around(image, coordinates, color):
    for i, v in coordinates.items():
        cv2.circle(image, (int(v[1][0]), int(v[1][1])), v[2], color, 2)
        cv2.circle(image, (int(v[1][0]), int(v[1][1])), 0, color, 5)

In [20]:
# ADD TEXT TO THE IMAGE

def add_Text(image, coordinates):
    i = 0
    for k, v in coordinates.items():
        i += 1
        cv2.putText(image, "#{}".format(i), (int(v[1][0]+50), int(v[1][1]) - 35), cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0, 255, 0), thickness=2)

### Read image #2 


In [21]:
# Select image for comparison and select csv-file with initial data about image #1

filename2 = open_file()
o_file = open_file()

In [22]:
filename2
o_file

('C:/Users/OKa/Documents/_OKa/III. JAKSO/4. DS project/HS1/2018.08.24.png',)

('C:/Users/OKa/Documents/_OKa/III. JAKSO/4. DS project/outputCSV.csv',)

In [23]:
im2 = cv2.imread(filename2[0], 1)
df = pd.read_csv(o_file[0], sep=';')
    
# Get rescaled and cropped images in color and gray
im2, im2_gray = scale_crop_gray(im2)

In [24]:
im2_COORDINATES = find_contours(filename2, im2, im2_gray)

Contours: 8


In [35]:
df

,ImageID,Image,"(X,Y) coordinates",Radius,Lenght change
0,1,2018.07.24.png,"(43, 51)",61,0


In [25]:
im2_COORDINATES

{1: ['2018.08.24.png', (333, 81), 334, 0],
 2: ['2018.08.24.png', (114, 6), 11, 0],
 3: ['2018.08.24.png', (186, 277), 24, 0],
 4: ['2018.08.24.png', (228, 199), 30, 0],
 5: ['2018.08.24.png', (371, 109), 33, 0],
 6: ['2018.08.24.png', (456, 136), 134, 0],
 7: ['2018.08.24.png', (702, 98), 93, 0],
 8: ['2018.08.24.png', (1055, 96), 129, 0]}

In [26]:
df

,ImageID,Image,"(X,Y) coordinates",Radius,Lenght change
0,1,2018.07.24.png,"(43, 51)",61,0


In [27]:
# Create dataframe for image #2

image2 = pd.DataFrame.from_dict(im2_COORDINATES, orient='index', columns=['Image','(X,Y) coordinates', 'Radius','Lenght change'])

In [28]:
image2

,Image,"(X,Y) coordinates",Radius,Lenght change
1,2018.08.24.png,"(333, 81)",334,0
2,2018.08.24.png,"(114, 6)",11,0
3,2018.08.24.png,"(186, 277)",24,0
4,2018.08.24.png,"(228, 199)",30,0
5,2018.08.24.png,"(371, 109)",33,0
6,2018.08.24.png,"(456, 136)",134,0
7,2018.08.24.png,"(702, 98)",93,0
8,2018.08.24.png,"(1055, 96)",129,0


In [33]:
# Auxiliary functions

def check_Radius(r_1, r_2):
    diff = r_2-r_1
    if diff > -5:
        print('Radius #1 < Radius #2')
        return True
    else:
        # print('Radius #1 >= Radius #2')
        return False

# check_distance() function to calculate distance between two points    
def check_distance(crd1, crd2):
    distance = math.sqrt(((crd1[0]-crd2[0])**2)+((crd1[1]-crd2[1])**2))
    if distance < 30:
        print(f"Distance between points: {distance} --> same root")
        return True
    else:
        print(f"Distance between points: {distance} --> different root")
        return False

# get image ID, coordinates and radius (IMAGE #1)
def extract_data_DF(df):
    return df['ImageID'], eval(df['(X,Y) coordinates']), df['Radius']

# get image coordinates from dictionary (IMAGE #2)
def extract_crd_DICT(nbr, dct):
    return dct[nbr][1]

# draw only one circle
def draw_circles_around_ONE(image, coordinates, color):
    cv2.circle(image, (coordinates[1][0], coordinates[1][1]), coordinates[2], color, 2)
    cv2.circle(image, (coordinates[1][0], coordinates[1][1]), 0, color, 5)
        
def add_Text_ONE(image, coordinates, idRoot):
    cv2.putText(image, "#{}".format(idRoot), (coordinates[1][0]+50, coordinates[1][1]-35), cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0, 255, 0), thickness=2)

In [29]:
# Create empty dataframe 

image = pd.DataFrame(index=image2.index, columns=image2.columns)

In [30]:
image

,Image,"(X,Y) coordinates",Radius,Lenght change
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN


In [31]:
def checkNextDayImage(im1_crd, im2_crd, im1_r, im2_r, im1_id, image, image1, image2):
    # Check if radius is bigger or not and get differentce (= root growing change)
    # if status is True --> same root by radious
    diff = im2_r - im1_r
    if check_Radius(im1_r, im2_r) and check_distance(im1_crd, im2_crd):
        print("True")
        image.loc[im1_id, :] = image2.loc[im1_id, :]
        image.loc[im1_id, 'Lenght change'] = diff
    else:
        print('False')


In [36]:
for i in range(len(im2_COORDINATES)):
    im2_id = i+1
    im2_crd = im2_COORDINATES[i+1][1]
    im2_r = im2_COORDINATES[i+1][2]
    print(f'Image #2: ID {im2_id}, coordinates {im2_crd} and radius {im2_r}')
    for j in range(df.shape[0]): 
        # imX_id is root number, imX_crd are root coordinations and imX_r is root radius
        (im1_id, im1_crd,im1_r) = extract_data_DF(df.iloc[j,:])
        print(f'Image #1: ID {im1_id}, coordinates {im1_crd} and radius {im1_r}')
        checkNextDayImage(im1_crd, im2_crd, im1_r, im2_r, im1_id, image, df, image2)

Image #2: ID 1, coordinates (333, 81) and radius 334
Image #1: ID 1, coordinates (43, 51) and radius 61
Radius #1 < Radius #2
Distance between points: 291.547594742265 --> different root
False
Image #2: ID 2, coordinates (114, 6) and radius 11
Image #1: ID 1, coordinates (43, 51) and radius 61
False
Image #2: ID 3, coordinates (186, 277) and radius 24
Image #1: ID 1, coordinates (43, 51) and radius 61
False
Image #2: ID 4, coordinates (228, 199) and radius 30
Image #1: ID 1, coordinates (43, 51) and radius 61
False
Image #2: ID 5, coordinates (371, 109) and radius 33
Image #1: ID 1, coordinates (43, 51) and radius 61
False
Image #2: ID 6, coordinates (456, 136) and radius 134
Image #1: ID 1, coordinates (43, 51) and radius 61
Radius #1 < Radius #2
Distance between points: 421.6562581060549 --> different root
False
Image #2: ID 7, coordinates (702, 98) and radius 93
Image #1: ID 1, coordinates (43, 51) and radius 61
Radius #1 < Radius #2
Distance between points: 660.6738983795258 --> di

In [37]:
image

,Image,"(X,Y) coordinates",Radius,Lenght change
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN


In [38]:
# drop empty rows

image.dropna(how='all')

,Image,"(X,Y) coordinates",Radius,Lenght change
